# 02 — CRUD Operations

Iceberg supports true row-level updates — unlike Hive tables where you'd have to rewrite entire partitions.

In this notebook:
1. INSERT more rows
2. UPDATE existing rows
3. DELETE rows
4. MERGE INTO (upsert)

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("IcebergDemo")
    .master("local[*]")
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1")
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.type", "hadoop")
    .config("spark.sql.catalog.demo.warehouse", "../warehouse")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .getOrCreate()
)
print("Spark + Iceberg ready.")

26/02/23 13:54:23 WARN Utils: Your hostname, barkha-xg1 resolves to a loopback address: 127.0.1.1; using 192.168.1.227 instead (on interface enp195s0)
26/02/23 13:54:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/barkha/.ivy2/cache
The jars for the packages stored in: /home/barkha/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cc68a922-4d0f-455f-88d1-6edb18fe8ae0;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 in central
:: resolution report :: resolve 52ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	----

:: loading settings :: url = jar:file:/home/barkha/iceberg-demo/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	0 artifacts copied, 1 already retrieved (0kB/2ms)
26/02/23 13:54:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/23 13:54:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark + Iceberg ready.


## Current state of the table

In [2]:
spark.sql("SELECT * FROM demo.ecommerce.orders ORDER BY order_id").show()

26/02/23 13:54:42 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------+--------+----------+--------+------+----------+
|order_id|customer|   product|quantity| price|order_date|
+--------+--------+----------+--------+------+----------+
|       1|   Alice|    Laptop|       1|999.99|2024-01-15|
|       2|     Bob|     Mouse|       2| 29.99|2024-01-16|
|       3| Charlie|  Keyboard|       1| 79.99|2024-01-16|
|       4|   Alice|   Monitor|       1|349.99|2024-01-17|
|       5|   Diana|Headphones|       3| 59.99|2024-01-18|
+--------+--------+----------+--------+------+----------+



## 1. INSERT — Add More Orders

In [3]:
spark.sql("""
    INSERT INTO demo.ecommerce.orders VALUES
        (6,  'Eve',   'Webcam',   1, 89.99,  DATE '2024-01-19'),
        (7,  'Bob',   'USB Hub',  2, 24.99,  DATE '2024-01-20'),
        (8,  'Charlie', 'Laptop', 1, 1099.99, DATE '2024-02-01')
""")

spark.sql("SELECT * FROM demo.ecommerce.orders ORDER BY order_id").show()

+--------+--------+----------+--------+-------+----------+
|order_id|customer|   product|quantity|  price|order_date|
+--------+--------+----------+--------+-------+----------+
|       1|   Alice|    Laptop|       1| 999.99|2024-01-15|
|       2|     Bob|     Mouse|       2|  29.99|2024-01-16|
|       3| Charlie|  Keyboard|       1|  79.99|2024-01-16|
|       4|   Alice|   Monitor|       1| 349.99|2024-01-17|
|       5|   Diana|Headphones|       3|  59.99|2024-01-18|
|       6|     Eve|    Webcam|       1|  89.99|2024-01-19|
|       7|     Bob|   USB Hub|       2|  24.99|2024-01-20|
|       8| Charlie|    Laptop|       1|1099.99|2024-02-01|
+--------+--------+----------+--------+-------+----------+



## 2. UPDATE — Fix a Price

Oops — Alice's laptop was supposed to be $899.99, not $999.99.

In [4]:
spark.sql("""
    UPDATE demo.ecommerce.orders
    SET price = 899.99
    WHERE order_id = 1
""")

spark.sql("SELECT * FROM demo.ecommerce.orders WHERE order_id = 1").show()

+--------+--------+-------+--------+------+----------+
|order_id|customer|product|quantity| price|order_date|
+--------+--------+-------+--------+------+----------+
|       1|   Alice| Laptop|       1|899.99|2024-01-15|
+--------+--------+-------+--------+------+----------+



## 3. DELETE — Remove a Cancelled Order

Eve cancelled her webcam order.

In [5]:
spark.sql("""
    DELETE FROM demo.ecommerce.orders
    WHERE order_id = 6
""")

spark.sql("SELECT * FROM demo.ecommerce.orders ORDER BY order_id").show()

+--------+--------+----------+--------+-------+----------+
|order_id|customer|   product|quantity|  price|order_date|
+--------+--------+----------+--------+-------+----------+
|       1|   Alice|    Laptop|       1| 899.99|2024-01-15|
|       2|     Bob|     Mouse|       2|  29.99|2024-01-16|
|       3| Charlie|  Keyboard|       1|  79.99|2024-01-16|
|       4|   Alice|   Monitor|       1| 349.99|2024-01-17|
|       5|   Diana|Headphones|       3|  59.99|2024-01-18|
|       7|     Bob|   USB Hub|       2|  24.99|2024-01-20|
|       8| Charlie|    Laptop|       1|1099.99|2024-02-01|
+--------+--------+----------+--------+-------+----------+



## 4. MERGE INTO — Upsert New Data

`MERGE INTO` lets you do an "upsert": update rows that match, insert rows that don't.

This is one of Iceberg's most powerful features for incremental data loading.

In [6]:
# Create a temporary view with incoming data
spark.sql("""
    CREATE OR REPLACE TEMP VIEW incoming_orders AS
    SELECT * FROM VALUES
        (2,  'Bob',   'Mouse',      2,  24.99,  DATE '2024-01-16'),  -- updated price
        (9,  'Diana', 'Mouse',      1,  29.99,  DATE '2024-02-02'),  -- new order
        (10, 'Frank', 'Keyboard',   2,  79.99,  DATE '2024-02-03')   -- new order
    AS t(order_id, customer, product, quantity, price, order_date)
""")

spark.sql("""
    MERGE INTO demo.ecommerce.orders target
    USING incoming_orders source
    ON target.order_id = source.order_id
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
""")

spark.sql("SELECT * FROM demo.ecommerce.orders ORDER BY order_id").show()

+--------+--------+----------+--------+-------+----------+
|order_id|customer|   product|quantity|  price|order_date|
+--------+--------+----------+--------+-------+----------+
|       1|   Alice|    Laptop|       1| 899.99|2024-01-15|
|       2|     Bob|     Mouse|       2|  24.99|2024-01-16|
|       3| Charlie|  Keyboard|       1|  79.99|2024-01-16|
|       4|   Alice|   Monitor|       1| 349.99|2024-01-17|
|       5|   Diana|Headphones|       3|  59.99|2024-01-18|
|       7|     Bob|   USB Hub|       2|  24.99|2024-01-20|
|       8| Charlie|    Laptop|       1|1099.99|2024-02-01|
|       9|   Diana|     Mouse|       1|  29.99|2024-02-02|
|      10|   Frank|  Keyboard|       2|  79.99|2024-02-03|
+--------+--------+----------+--------+-------+----------+



## Key Takeaway

| Operation  | Traditional Hive           | Iceberg                    |
|------------|----------------------------|----------------------------|
| INSERT     | Supported                  | Supported                  |
| UPDATE     | Rewrite entire partition   | Row-level update           |
| DELETE     | Rewrite entire partition   | Row-level delete           |
| MERGE INTO | Not natively supported     | Built-in upsert support    |

Every operation we just ran created a **new snapshot** — we'll explore those in the next notebook!

**Next up:** Time travel in notebook 03!